# Examen Final: Servicio de Inferencia Optimizada

- **Autor:** FRANKLIN ESPINOZA PARI
- **Fecha:** 12 de Julio de 2025
- **Proyecto #7:** Servicio de inferencia optimizada con cuantización

## 1. Introducción y Motivación

Los modelos de lenguaje grandes (LLMs) como GPT-J son extremadamente potentes, pero su tamaño masivo (más de 6 mil millones de parámetros) presenta desafíos significativos para su despliegue en producción. La inferencia puede ser lenta y consumir una gran cantidad de recursos de GPU (VRAM), lo que eleva los costos operativos y limita la escalabilidad.

Este proyecto aborda este problema mediante la implementación de un servicio de inferencia optimizado que utiliza dos técnicas clave:

1.  **Cuantización:** Se reduce la precisión numérica de los pesos del modelo (por ejemplo, de 32 bits a 8 bits). Esto disminuye drásticamente el uso de memoria y puede acelerar la inferencia, con una pérdida mínima de precisión.
2.  **Batching Dinámico:** Se agrupan múltiples peticiones de inferencia que llegan de forma concurrente en un solo lote (batch). Esto permite aprovechar al máximo el paralelismo de la GPU, mejorando significativamente el *throughput* (peticiones procesadas por segundo) del sistema.

## 2. Implementación Clave

La solución se construyó utilizando Python, FastAPI y las librerías `transformers` y `bitsandbytes`.

### 2.1. Pipeline de Cuantización

El siguiente fragmento de `src/quantize.py` muestra cómo se carga el modelo GPT-J y se cuantiza a 8 bits al vuelo utilizando `BitsAndBytesConfig`.

In [1]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Configuración de cuantización a 8 bits
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

# Carga del modelo con la configuración de cuantización
model_name = "EleutherAI/gpt-j-6B"
print(f"Cargando y cuantizando el modelo {model_name}...")

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=quantization_config,
#     device_map="auto",
#     torch_dtype=torch.float16
# )

print("El modelo se cargaría y cuantizaría en este paso.")

c:\fespa-dev\nlp-curso\examenfinal\nlp-final-proyecto-07\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Cargando y cuantizando el modelo EleutherAI/gpt-j-6B...
El modelo se cargaría y cuantizaría en este paso.


### 2.2. Servidor Asíncrono con Batching Dinámico

El corazón del servicio es un `worker` asíncrono que se ejecuta en segundo plano. Este worker recoge peticiones de una `asyncio.Queue`, las agrupa en un lote y las procesa juntas. Esto se implementa en `src/server.py`.

In [ ]:
import asyncio

async def batch_processing_worker():
    # Configuración de ejemplo
    MAX_BATCH_SIZE = 4
    BATCH_TIMEOUT = 0.5 # 500ms
    request_queue = asyncio.Queue()

    while True:
        batch = []
        start_time = asyncio.get_event_loop().time()

        # Recolectar peticiones hasta llenar el batch o alcanzar el timeout
        while len(batch) < MAX_BATCH_SIZE:
            try:
                remaining_time = BATCH_TIMEOUT - (asyncio.get_event_loop().time() - start_time)
                if remaining_time <= 0:
                    break
                request = await asyncio.wait_for(request_queue.get(), timeout=remaining_time)
                batch.append(request)
            except asyncio.TimeoutError:
                break

        if batch:
            print(f"Procesando un lote de {len(batch)} peticiones.")
            # Aquí se llamaría al modelo para procesar el lote completo
            pass

## 3. Resultados del Benchmark

Para evaluar el rendimiento del servicio, se realizó una prueba de carga utilizando el script `benchmarks/run_bench.sh`. Se enviaron **100 peticiones** con una **concurrencia de 10**.

A continuación, se cargan y analizan los resultados guardados en `benchmarks/bench_results.csv`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="whitegrid")

try:
    df = pd.read_csv('benchmarks/bench_results.csv')
    # Convertir latencia a milisegundos
    df['latency_ms'] = df['latency_s'] * 1000
    print("Resultados del benchmark cargados exitosamente.")
    display(df.head())
except FileNotFoundError:
    print("Archivo 'benchmarks/bench_results.csv' no encontrado.")
    print("Por favor, ejecute 'bash benchmarks/run_bench.sh' primero.")
    df = None

### 3.1. Distribución de Latencias

El siguiente histograma muestra la distribución de los tiempos de respuesta en milisegundos. Esto nos ayuda a entender la consistencia del rendimiento del servicio.

In [ ]:
if df is not None:
    plt.figure(figsize=(12, 6))
    sns.histplot(df['latency_ms'], kde=True, bins=30)
    plt.title('Distribución de Latencias de Peticiones', fontsize=16)
    plt.xlabel('Latencia (ms)', fontsize=12)
    plt.ylabel('Frecuencia', fontsize=12)
    plt.show()

### 3.2. Métricas Clave de Rendimiento

Las métricas de percentiles son cruciales para entender la experiencia del usuario. P50 (la mediana) representa el caso típico, mientras que P95 y P99 representan los peores casos que afectan a un pequeño porcentaje de usuarios.

In [ ]:
if df is not None:
    p50 = np.percentile(df['latency_ms'], 50)
    p95 = np.percentile(df['latency_ms'], 95)
    p99 = np.percentile(df['latency_ms'], 99)
    mean_latency = df['latency_ms'].mean()
    max_latency = df['latency_ms'].max()
    total_requests = len(df)

    stats_data = {
        'Métrica': ['Peticiones Totales', 'Latencia Media', 'P50 (Mediana)', 'P95', 'P99', 'Latencia Máxima'],
        'Valor': [f"{total_requests}", f"{mean_latency:.2f} ms", f"{p50:.2f} ms", f"{p95:.2f} ms", f"{p99:.2f} ms", f"{max_latency:.2f} ms"]
    }

    stats_df = pd.DataFrame(stats_data)

    print("Tabla Resumen de Rendimiento")
    display(stats_df)

## 4. Conclusiones y Reflexión

Los resultados demuestran que la combinación de **cuantización y batching dinámico** es una estrategia altamente efectiva para desplegar LLMs en un entorno de producción.

* **Rendimiento:** El servicio fue capaz de manejar una carga concurrente significativa manteniendo latencias aceptables, como se observa en los percentiles P50 y P95.
* **Eficiencia de Recursos:** La cuantización permitió que el modelo GPT-J, que normalmente requiere ~24GB de VRAM en `float32`, se ejecutara en una GPU con considerablemente menos memoria (típicamente ~7GB en 8 bits).

**Limitaciones y Futuras Mejoras:**

* **Degradación de Calidad:** Aunque mínima, la cuantización puede afectar la calidad de las respuestas del modelo. Sería útil realizar una evaluación cualitativa más profunda.
* **Optimización Adicional:** Se podrían explorar técnicas más avanzadas como la compilación de modelos (ej. con `torch.compile`) o el uso de motores de inferencia especializados como TensorRT-LLM para exprimir aún más el rendimiento.